In [1]:
import numpy as np
import time
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from dgl import model_zoo,DGLGraph
import math
import networkx as nx
import matplotlib.pyplot as plt
#from utils_mpnn import Meter, set_random_seed, collate, EarlyStopping, load_model,load_brl_dataset,regress,run_a_train_epoch,run_an_eval_epoch
import argparse
# from sklearn import svm, datasets
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import plot_confusion_matrix
import seaborn as sn
import pandas as pd
from sklearn import metrics
import collections
import torch.nn as nn
import torch.nn.functional as F
from utils_mlp import Meter

# define NN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        
        self.fc1 = nn.Linear(365*7+7, 4096)
        self.fc2 = nn.Linear(4096,4096)
        self.fc3 = nn.Linear(4096,4096)
        self.fc4 = nn.Linear(4096,4096)
        self.fc5_1 = nn.Linear(4096,4096)
        self.fc5_2 = nn.Linear(4096,4096)
        self.fc_ouput1 = nn.Linear(4096,121)
        self.fc_output2 = nn.Linear(4096,120)
        self.droput = nn.Dropout(0.1)
        self.sig=nn.Sigmoid()
        
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = self.droput(x)
        x = F.relu(self.fc2(x))
        x = self.droput(x)
        x = F.relu(self.fc3(x))
        x = self.droput(x)
        x = F.relu(self.fc4(x))
        x = self.droput(x)
        
        x1 = F.relu(self.fc5_1(x))
        x2 = F.relu(self.fc5_2(x))
        
        x1 = self.fc_ouput1(x1)
        x2 = self.sig(self.fc_output2(x2))#x2 = self.fc4_2(x)
        return [x1,x2]#x
# initialize the NN
model = Net()

print(torch.cuda.current_device())
torch.cuda.set_device(1)
print(torch.cuda.current_device())

loss_fn_cn =nn.BCEWithLogitsLoss(reduction='none')
loss_fn_a =nn.L1Loss(reduction='none')
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#stopper = EarlyStopping(mode='lower', patience=500, filename='model_saved/kld/early_stop.pth')
args={};args['device']=torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(args['device'])

class MyDataset(Dataset):
    def __init__(self, data, target_cn, target_a,transform=None):
        self.data = torch.from_numpy(data).float()
        self.target_cn = torch.from_numpy(target_cn).float()#.long()
        self.target_a = torch.from_numpy(target_a).float()
        self.transform = transform
    def __getitem__(self, index):
        x = self.data[index]
        y_cn = self.target_cn[index]
        y_a = self.target_a[index]
        if self.transform:
            x = self.transform(x)
        return x, y_cn, y_a
    def __len__(self):
        return len(self.data)

data_path_x='data/data_mlp/input365/'
data_path_label_c='data/data_mlp/label_multitask_bce/';data_path_label_a='data/data_mlp/label/'
mode='train'
dataset = MyDataset(np.load(data_path_x+'features_'+mode+'.npz')['arr_0'], 
                    np.load(data_path_label_c+'c_label_'+mode+'.npz')['arr_0'],
                    np.load(data_path_label_a+'label_'+mode+'.npz')['arr_0'])
train_loader = torch.utils.data.DataLoader(dataset, batch_size=512, shuffle=True,pin_memory=True)

mode='val'
dataset = MyDataset(np.load(data_path_x+'features_'+mode+'.npz')['arr_0'], 
                    np.load(data_path_label_c+'c_label_'+mode+'.npz')['arr_0'], 
                    np.load(data_path_label_a+'label_'+mode+'.npz')['arr_0'])
val_loader = torch.utils.data.DataLoader(dataset, batch_size=512, shuffle=False,pin_memory=True)

mode='test'
dataset = MyDataset(np.load(data_path_x+'features_'+mode+'.npz')['arr_0'],
                    np.load(data_path_label_c+'c_label_'+mode+'.npz')['arr_0'],
                    np.load(data_path_label_a+'label_'+mode+'.npz')['arr_0'])
test_loader = torch.utils.data.DataLoader(dataset, batch_size=512, shuffle=False,pin_memory=True)

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf  # set initial "min" to infinity

Using backend: pytorch
/home/shs/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


0
1


Epoch: 1 	Training Loss: 99.188858 	acc: 98.398760	Validation Loss: 38.934124, time5.4
Validation loss decreased (inf --> 98.398760).  Saving model ...
Epoch: 2 	Training Loss: 99.007117 	acc: 98.679982	Validation Loss: 38.534096, time5.2
Epoch: 3 	Training Loss: 99.072161 	acc: 99.276860	Validation Loss: 37.878101, time5.2
Epoch: 4 	Training Loss: 99.075987 	acc: 98.886593	Validation Loss: 39.178919, time5.3
Epoch: 5 	Training Loss: 98.976507 	acc: 98.949725	Validation Loss: 38.675041, time5.2
Epoch: 6 	Training Loss: 98.930594 	acc: 99.265381	Validation Loss: 40.346725, time5.3
Epoch: 7 	Training Loss: 98.833027 	acc: 98.966942	Validation Loss: 34.509078, time5.3
Epoch: 8 	Training Loss: 98.616850 	acc: 98.329890	Validation Loss: 39.044914, time5.3
Validation loss decreased (98.398760 --> 98.329890).  Saving model ...
Epoch: 9 	Training Loss: 98.551806 	acc: 98.433196	Validation Loss: 35.877952, time5.2
Epoch: 10 	Training Loss: 98.490588 	acc: 98.364325	Validation Loss: 38.608404, t

Epoch: 73 	Training Loss: 90.524564 	acc: 90.570478	Validation Loss: 38.788853, time5.5
Epoch: 74 	Training Loss: 90.260560 	acc: 91.018136	Validation Loss: 40.045322, time5.4
Epoch: 75 	Training Loss: 90.241429 	acc: 90.639348	Validation Loss: 40.075926, time5.5
Epoch: 76 	Training Loss: 90.008035 	acc: 90.633609	Validation Loss: 39.281137, time5.5
Epoch: 77 	Training Loss: 89.952556 	acc: 90.576217	Validation Loss: 39.386281, time5.5
Epoch: 78 	Training Loss: 89.862642 	acc: 89.726814	Validation Loss: 40.540902, time5.5
Validation loss decreased (90.495868 --> 89.726814).  Saving model ...
Epoch: 79 	Training Loss: 89.778466 	acc: 90.249082	Validation Loss: 39.854123, time5.3
Epoch: 80 	Training Loss: 89.506811 	acc: 90.185950	Validation Loss: 39.095801, time5.3
Epoch: 81 	Training Loss: 89.590986 	acc: 90.518825	Validation Loss: 39.737841, time5.3
Epoch: 82 	Training Loss: 89.353765 	acc: 90.082645	Validation Loss: 39.931571, time5.3
Epoch: 83 	Training Loss: 89.198806 	acc: 89.5087

Epoch: 157 	Training Loss: 81.330349 	acc: 87.913223	Validation Loss: 54.967624, time5.5
Epoch: 158 	Training Loss: 81.047215 	acc: 88.871671	Validation Loss: 55.089102, time5.4
Epoch: 159 	Training Loss: 81.219391 	acc: 87.396694	Validation Loss: 56.180354, time5.4
Validation loss decreased (87.609045 --> 87.396694).  Saving model ...
Epoch: 160 	Training Loss: 81.073998 	acc: 88.297750	Validation Loss: 56.098585, time5.6
Epoch: 161 	Training Loss: 80.842516 	acc: 88.653581	Validation Loss: 56.634966, time5.5
Epoch: 162 	Training Loss: 80.915213 	acc: 87.947658	Validation Loss: 55.964241, time5.5
Epoch: 163 	Training Loss: 80.637818 	acc: 87.620523	Validation Loss: 56.857614, time5.5
Epoch: 164 	Training Loss: 80.586165 	acc: 88.487144	Validation Loss: 57.082041, time5.5
Epoch: 165 	Training Loss: 80.679905 	acc: 87.844353	Validation Loss: 57.246802, time5.5
Epoch: 166 	Training Loss: 80.511555 	acc: 87.551653	Validation Loss: 57.724228, time5.5
Epoch: 167 	Training Loss: 80.157637 	a

Epoch: 247 	Training Loss: 73.953551 	acc: 87.511478	Validation Loss: 77.404719, time5.5
Epoch: 248 	Training Loss: 73.953551 	acc: 87.798439	Validation Loss: 76.019036, time5.4
Epoch: 249 	Training Loss: 73.649373 	acc: 87.729568	Validation Loss: 75.765388, time5.5
Epoch: 250 	Training Loss: 73.622590 	acc: 88.842975	Validation Loss: 77.182660, time5.4
Epoch: 251 	Training Loss: 73.660851 	acc: 89.124197	Validation Loss: 75.927355, time5.4
Epoch: 252 	Training Loss: 73.704852 	acc: 88.154270	Validation Loss: 77.454641, time5.5
Epoch: 253 	Training Loss: 73.408326 	acc: 88.974977	Validation Loss: 77.494792, time5.4
Epoch: 254 	Training Loss: 73.551806 	acc: 88.607668	Validation Loss: 77.370843, time5.5
Epoch: 255 	Training Loss: 73.503979 	acc: 87.591827	Validation Loss: 78.611247, time5.4
Epoch: 256 	Training Loss: 73.182583 	acc: 88.653581	Validation Loss: 79.466225, time5.5
Epoch: 257 	Training Loss: 73.433196 	acc: 87.660698	Validation Loss: 78.548445, time5.4
Epoch: 258 	Training 

Epoch: 340 	Training Loss: 68.973829 	acc: 88.223140	Validation Loss: 92.457582, time5.4
Epoch: 341 	Training Loss: 68.700260 	acc: 87.970615	Validation Loss: 93.943517, time5.2
Epoch: 342 	Training Loss: 69.197658 	acc: 88.050964	Validation Loss: 93.731002, time5.3
Epoch: 343 	Training Loss: 68.878176 	acc: 88.630624	Validation Loss: 93.359552, time5.2
Epoch: 344 	Training Loss: 68.436257 	acc: 88.125574	Validation Loss: 93.285453, time5.2
Epoch: 345 	Training Loss: 68.593128 	acc: 88.521579	Validation Loss: 92.499657, time5.3
Epoch: 346 	Training Loss: 68.906872 	acc: 88.710973	Validation Loss: 93.421481, time5.2
Epoch: 347 	Training Loss: 68.675390 	acc: 88.332185	Validation Loss: 95.079787, time5.3
Epoch: 348 	Training Loss: 68.784435 	acc: 89.101240	Validation Loss: 92.901893, time5.2
Epoch: 349 	Training Loss: 68.849480 	acc: 88.860193	Validation Loss: 93.934056, time5.3
Epoch: 350 	Training Loss: 68.126339 	acc: 88.865932	Validation Loss: 94.136438, time5.2
Epoch: 351 	Training 

...

Epoch: 2421 	Training Loss: 55.614861 	acc: 86.208678	Validation Loss: 192.735066, time5.3
Epoch: 2422 	Training Loss: 55.392945 	acc: 86.398072	Validation Loss: 198.715649, time5.3
Epoch: 2423 	Training Loss: 55.362335 	acc: 85.910239	Validation Loss: 198.049875, time5.3
Epoch: 2424 	Training Loss: 55.364249 	acc: 86.139807	Validation Loss: 199.933449, time5.3
Epoch: 2425 	Training Loss: 55.597643 	acc: 86.771120	Validation Loss: 188.481557, time5.2
Epoch: 2426 	Training Loss: 55.716253 	acc: 86.237374	Validation Loss: 203.779688, time5.3
Epoch: 2427 	Training Loss: 55.375727 	acc: 86.289027	Validation Loss: 199.399483, time5.3
Epoch: 2428 	Training Loss: 55.366162 	acc: 86.564509	Validation Loss: 198.897483, time5.3
Epoch: 2429 	Training Loss: 55.957300 	acc: 86.644858	Validation Loss: 193.361323, time5.3
Epoch: 2430 	Training Loss: 55.574686 	acc: 86.598944	Validation Loss: 198.467250, time5.3
Epoch: 2431 	Training Loss: 54.866850 	acc: 86.461203	Validation Loss: 196.541919, time5.3

Epoch: 2512 	Training Loss: 54.176232 	acc: 85.818411	Validation Loss: 208.520374, time5.3
Epoch: 2513 	Training Loss: 55.352770 	acc: 86.352158	Validation Loss: 201.685711, time5.3
Epoch: 2514 	Training Loss: 56.433655 	acc: 86.340680	Validation Loss: 199.768848, time5.3
Epoch: 2515 	Training Loss: 54.864937 	acc: 86.466942	Validation Loss: 206.188695, time5.3
Epoch: 2516 	Training Loss: 53.942837 	acc: 86.260331	Validation Loss: 215.452692, time5.3
Epoch: 2517 	Training Loss: 54.685109 	acc: 86.277548	Validation Loss: 205.160405, time5.3
Epoch: 2518 	Training Loss: 54.681283 	acc: 87.017906	Validation Loss: 209.294229, time5.3
Epoch: 2519 	Training Loss: 55.329813 	acc: 86.438246	Validation Loss: 203.120467, time5.3
Epoch: 2520 	Training Loss: 54.895546 	acc: 86.122590	Validation Loss: 204.384011, time5.3
Epoch: 2521 	Training Loss: 54.348408 	acc: 87.482782	Validation Loss: 204.220412, time5.2
Epoch: 2522 	Training Loss: 55.083027 	acc: 87.006428	Validation Loss: 208.760322, time5.3

Epoch: 2603 	Training Loss: 54.574151 	acc: 85.715106	Validation Loss: 212.462720, time5.3
Epoch: 2604 	Training Loss: 55.113636 	acc: 88.452709	Validation Loss: 203.030998, time5.3
Epoch: 2605 	Training Loss: 55.230334 	acc: 86.082415	Validation Loss: 208.957007, time5.3
Epoch: 2606 	Training Loss: 54.790328 	acc: 87.029385	Validation Loss: 209.895281, time5.3
Epoch: 2607 	Training Loss: 55.616774 	acc: 86.759642	Validation Loss: 212.352333, time5.2
Epoch: 2608 	Training Loss: 54.587542 	acc: 86.943297	Validation Loss: 211.505896, time5.3
Epoch: 2609 	Training Loss: 55.105984 	acc: 86.289027	Validation Loss: 209.608305, time5.3
Epoch: 2610 	Training Loss: 54.220233 	acc: 86.380854	Validation Loss: 205.473877, time5.3
Epoch: 2611 	Training Loss: 54.884068 	acc: 86.713728	Validation Loss: 207.706511, time5.3
Epoch: 2612 	Training Loss: 54.931895 	acc: 86.093893	Validation Loss: 212.414781, time5.3
Epoch: 2613 	Training Loss: 54.430670 	acc: 86.644858	Validation Loss: 212.458937, time5.3

Epoch: 2694 	Training Loss: 55.060070 	acc: 85.783976	Validation Loss: 214.700440, time5.3
Epoch: 2695 	Training Loss: 54.857285 	acc: 85.841368	Validation Loss: 209.389114, time5.3
Epoch: 2696 	Training Loss: 55.241812 	acc: 86.484160	Validation Loss: 211.130274, time5.3
Epoch: 2697 	Training Loss: 55.352770 	acc: 86.857208	Validation Loss: 202.085222, time5.3
Epoch: 2698 	Training Loss: 56.334175 	acc: 87.534435	Validation Loss: 211.771097, time5.3
Epoch: 2699 	Training Loss: 55.676079 	acc: 85.893021	Validation Loss: 209.071020, time5.3
Epoch: 2700 	Training Loss: 54.973982 	acc: 86.507117	Validation Loss: 206.436521, time5.3
Epoch: 2701 	Training Loss: 57.007576 	acc: 85.950413	Validation Loss: 208.061032, time5.3
Epoch: 2702 	Training Loss: 55.563208 	acc: 86.530073	Validation Loss: 210.032462, time5.3
Epoch: 2703 	Training Loss: 56.812443 	acc: 86.690771	Validation Loss: 199.302379, time5.3
Epoch: 2704 	Training Loss: 56.896618 	acc: 86.512856	Validation Loss: 207.758857, time5.2

KeyboardInterrupt: 

In [3]:
# torch.save({'model_state_dict': model.state_dict(),
#     'optimizer': optimizer.state_dict()}, 'model_saved/mlp/multitask/'+str(epoch)+'.pth')

state=torch.load('model_saved/mlp/multitask/model.pth') 
model.load_state_dict(state['model_state_dict']) 
optimizer.load_state_dict(state['optimizer'])

for param_group in optimizer.param_groups: print(param_group['lr'])
for param_group in optimizer.param_groups: param_group['lr'] = param_group['lr']*0.1
for param_group in optimizer.param_groups: print(param_group['lr'])

0.001
0.0001


In [4]:
#number of epochs to train the model
ep_list=[2000,4000,6000]
n_epochs = 3000
for epoch in range(n_epochs):
    st=time.time()
    '''if False:#epoch in ep_list:
        torch.save({'model_state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()}, 'model_saved/mlp_multitask/'+str(epoch)+'.pth')
        for param_group in optimizer.param_groups: print(param_group['lr'])
        for param_group in optimizer.param_groups: param_group['lr'] = param_group['lr']*0.1
        for param_group in optimizer.param_groups: print(param_group['lr'])'''
    # monitor losses
    train_loss = 0
    valid_loss = 0
    # train the model
    model.train() # prep model for training
    
    train_meter = Meter()
    for data,label_cn,label_a in train_loader:
        data = data.to(args['device'])
        label_cn = label_cn.to(args['device'])
        label_a = label_a.to(args['device'])
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        output_cn = output[0].to(args['device'])
        output_a = output[1].to(args['device'])
        # calculate the loss
        #loss = (loss_fn(output,label).float()).mean()
        loss_cn = (loss_fn_cn(output_cn, label_cn).float()).mean()
        ###loss_area = (loss_fn_a(output_a,label_a).float()).mean()
        loss_area1 = ((label_cn[:,1:]*loss_fn_a(output_a,label_a)).float()).mean()#/label_cn[:,1:].sum()
        loss_area2 = (((1-label_cn[:,1:])*loss_fn_a(output_a,label_a)).float()).mean()#/(1-label_cn[:,1:]).sum()
        loss=(5*loss_cn+120*loss_area1+loss_area2)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        _, output_cn = torch.max(output_cn.data, 1)
        _, label_cn = torch.max(label_cn.data, 1)
        train_meter.update(output_cn, label_cn)#train_meter.update(output_cn, loss_cn)
        
    eval_meter_cn = Meter()
    eval_meter_a = Meter()
    # validate the model #
    model.eval()  # prep model for evaluation
    for data,label_cn,label_a in val_loader:
        data = data.to(args['device'])
        label_cn = label_cn.to(args['device'])
        label_a = label_a.to(args['device'])
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        output_cn = output[0].to(args['device'])
        output_a = output[1].to(args['device'])
        # update running validation loss
        eval_meter_a.update(output_cn[:,1:]*output_a, output_cn[:,1:]*label_a)
        _, output_cn = torch.max(output_cn.data, 1)
        _, label_cn = torch.max(label_cn.data, 1)
        eval_meter_cn.update(output_cn, label_cn)
        #eval_meter_a.update(output_a, label_a)
    
    # print training/validation statistics # calculate average loss over an epoch
    train_loss=train_meter.compute_metric('acc')
    valid_loss_cn=eval_meter_cn.compute_metric('acc')
    valid_loss_a=np.sum(eval_meter_a.compute_metric('rmse'))
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tacc: {:.6f}\tValidation Loss: {:.6f}, time{:.1f}'.format(epoch+1, train_loss,valid_loss_cn,valid_loss_a,time.time()-st))
    
    # save model if validation loss has decreased
    if valid_loss_cn <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss_cn))
        #torch.save(model.state_dict(), 'model_saved/mlp_cn/model.pt')
        torch.save({'model_state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()}, 'model_saved/mlp/multitask/model.pth')
        
        valid_loss_min = valid_loss_cn
        
torch.save({'model_state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()}, 'model_saved/mlp/multitask/last.pth')

Epoch: 1 	Training Loss: 51.497934 	acc: 85.588843	Validation Loss: 201.576589, time5.2
Validation loss decreased (85.692149 --> 85.588843).  Saving model ...
Epoch: 2 	Training Loss: 51.084711 	acc: 85.542929	Validation Loss: 205.263144, time5.2
Validation loss decreased (85.588843 --> 85.542929).  Saving model ...
Epoch: 3 	Training Loss: 50.948883 	acc: 85.548669	Validation Loss: 207.247040, time5.2
Epoch: 4 	Training Loss: 50.397919 	acc: 85.594582	Validation Loss: 208.872700, time5.2
Epoch: 5 	Training Loss: 50.514616 	acc: 85.606061	Validation Loss: 211.116680, time5.3
Epoch: 6 	Training Loss: 50.572008 	acc: 85.548669	Validation Loss: 212.486968, time5.2
Epoch: 7 	Training Loss: 50.147306 	acc: 85.548669	Validation Loss: 213.634955, time5.3
Epoch: 8 	Training Loss: 50.327135 	acc: 85.542929	Validation Loss: 214.268986, time5.3
Validation loss decreased (85.542929 --> 85.542929).  Saving model ...
Epoch: 9 	Training Loss: 50.229568 	acc: 85.611800	Validation Loss: 215.519638, tim

Epoch: 89 	Training Loss: 49.022421 	acc: 85.709366	Validation Loss: 236.737421, time5.3
Epoch: 90 	Training Loss: 49.339991 	acc: 85.611800	Validation Loss: 237.221610, time5.3
Epoch: 91 	Training Loss: 49.324686 	acc: 85.629017	Validation Loss: 237.970879, time5.3
Epoch: 92 	Training Loss: 49.389731 	acc: 85.749541	Validation Loss: 237.458214, time5.4
Epoch: 93 	Training Loss: 49.144858 	acc: 85.583104	Validation Loss: 237.860057, time5.5
Epoch: 94 	Training Loss: 49.125727 	acc: 85.554408	Validation Loss: 238.239106, time5.4
Epoch: 95 	Training Loss: 48.823462 	acc: 85.606061	Validation Loss: 236.800661, time5.5
Epoch: 96 	Training Loss: 49.389731 	acc: 85.497016	Validation Loss: 238.078285, time5.5
Epoch: 97 	Training Loss: 49.334252 	acc: 85.537190	Validation Loss: 237.398318, time5.5
Epoch: 98 	Training Loss: 49.064509 	acc: 85.692149	Validation Loss: 235.904341, time5.5
Epoch: 99 	Training Loss: 49.242424 	acc: 85.525712	Validation Loss: 236.559409, time5.5
Epoch: 100 	Training 

Epoch: 180 	Training Loss: 48.863636 	acc: 85.674931	Validation Loss: 246.223032, time5.3
Epoch: 181 	Training Loss: 48.226584 	acc: 85.709366	Validation Loss: 244.669474, time5.2
Epoch: 182 	Training Loss: 48.798592 	acc: 85.669192	Validation Loss: 245.472221, time5.3
Epoch: 183 	Training Loss: 48.760331 	acc: 85.674931	Validation Loss: 245.177773, time5.3
Epoch: 184 	Training Loss: 48.681895 	acc: 85.611800	Validation Loss: 245.837872, time5.3
Epoch: 185 	Training Loss: 48.896159 	acc: 85.715106	Validation Loss: 244.837601, time5.3
Epoch: 186 	Training Loss: 48.842593 	acc: 85.709366	Validation Loss: 244.745372, time5.3
Epoch: 187 	Training Loss: 48.737374 	acc: 85.611800	Validation Loss: 245.181215, time5.2
Epoch: 188 	Training Loss: 48.863636 	acc: 85.629017	Validation Loss: 244.747394, time5.4
Epoch: 189 	Training Loss: 48.704852 	acc: 85.519972	Validation Loss: 244.486968, time5.3
Epoch: 190 	Training Loss: 48.789027 	acc: 85.680670	Validation Loss: 244.703749, time5.4
Epoch: 191

Epoch: 271 	Training Loss: 48.613024 	acc: 85.623278	Validation Loss: 250.599883, time5.4
Epoch: 272 	Training Loss: 48.576676 	acc: 85.629017	Validation Loss: 250.053728, time5.5
Epoch: 273 	Training Loss: 48.421717 	acc: 85.537190	Validation Loss: 250.132370, time5.5
Epoch: 274 	Training Loss: 48.276324 	acc: 85.651974	Validation Loss: 249.340296, time5.5
Epoch: 275 	Training Loss: 48.209366 	acc: 85.565886	Validation Loss: 250.554465, time5.5
Epoch: 276 	Training Loss: 48.561371 	acc: 85.824151	Validation Loss: 249.674629, time5.5
Epoch: 277 	Training Loss: 48.381543 	acc: 85.577365	Validation Loss: 251.053787, time5.5
Epoch: 278 	Training Loss: 48.542241 	acc: 85.692149	Validation Loss: 250.090981, time5.5
Epoch: 279 	Training Loss: 48.454239 	acc: 85.697888	Validation Loss: 250.803042, time5.5
Epoch: 280 	Training Loss: 48.327977 	acc: 85.812672	Validation Loss: 251.416510, time5.5
Epoch: 281 	Training Loss: 48.561371 	acc: 85.606061	Validation Loss: 250.241531, time5.4
Epoch: 282

Epoch: 363 	Training Loss: 48.303107 	acc: 85.600321	Validation Loss: 251.435419, time5.3
Epoch: 364 	Training Loss: 48.586241 	acc: 85.634757	Validation Loss: 251.694276, time5.3
Epoch: 365 	Training Loss: 48.165366 	acc: 85.583104	Validation Loss: 251.865300, time5.4
Epoch: 366 	Training Loss: 47.878405 	acc: 85.657713	Validation Loss: 251.806623, time5.5
Epoch: 367 	Training Loss: 48.048669 	acc: 85.502755	Validation Loss: 253.792244, time5.5
Epoch: 368 	Training Loss: 48.167279 	acc: 85.778237	Validation Loss: 253.080118, time5.4
Epoch: 369 	Training Loss: 48.679982 	acc: 85.433884	Validation Loss: 250.567208, time5.3
Epoch: 370 	Training Loss: 47.972146 	acc: 85.405188	Validation Loss: 251.282888, time5.3
Epoch: 371 	Training Loss: 48.148148 	acc: 85.588843	Validation Loss: 252.753646, time5.3
Epoch: 372 	Training Loss: 48.243802 	acc: 85.560147	Validation Loss: 250.417191, time5.3
Epoch: 373 	Training Loss: 47.899449 	acc: 85.485537	Validation Loss: 252.452239, time5.3
Epoch: 374

...

Epoch: 915 	Training Loss: 47.748316 	acc: 85.514233	Validation Loss: 263.996857, time5.5
Epoch: 916 	Training Loss: 47.551270 	acc: 85.680670	Validation Loss: 262.472872, time5.5
Epoch: 917 	Training Loss: 47.564662 	acc: 85.474059	Validation Loss: 264.200389, time5.5
Epoch: 918 	Training Loss: 47.694751 	acc: 85.548669	Validation Loss: 263.197133, time5.5
Epoch: 919 	Training Loss: 47.258571 	acc: 85.611800	Validation Loss: 263.550209, time5.5
Epoch: 920 	Training Loss: 47.721534 	acc: 85.376492	Validation Loss: 264.482229, time5.5
Epoch: 921 	Training Loss: 47.325528 	acc: 85.560147	Validation Loss: 266.146021, time5.5
Epoch: 922 	Training Loss: 47.291093 	acc: 85.422406	Validation Loss: 264.821190, time5.5
Epoch: 923 	Training Loss: 47.262397 	acc: 85.508494	Validation Loss: 264.995428, time5.5
Epoch: 924 	Training Loss: 47.356137 	acc: 85.674931	Validation Loss: 265.276741, time5.4
Epoch: 925 	Training Loss: 47.928145 	acc: 85.594582	Validation Loss: 264.184914, time5.5
Epoch: 926

Epoch: 1006 	Training Loss: 47.252831 	acc: 85.611800	Validation Loss: 266.654577, time5.5
Epoch: 1007 	Training Loss: 47.358050 	acc: 85.893021	Validation Loss: 265.356016, time5.5
Epoch: 1008 	Training Loss: 47.578053 	acc: 85.554408	Validation Loss: 265.733098, time5.5
Epoch: 1009 	Training Loss: 47.392485 	acc: 85.542929	Validation Loss: 266.968403, time5.5
Epoch: 1010 	Training Loss: 47.185874 	acc: 85.514233	Validation Loss: 265.049583, time5.4
Epoch: 1011 	Training Loss: 47.304484 	acc: 85.692149	Validation Loss: 265.792452, time5.5
Epoch: 1012 	Training Loss: 47.629706 	acc: 85.824151	Validation Loss: 265.418246, time5.5
Epoch: 1013 	Training Loss: 47.440312 	acc: 85.726584	Validation Loss: 264.130253, time5.5
Epoch: 1014 	Training Loss: 46.864478 	acc: 85.542929	Validation Loss: 265.669355, time5.5
Epoch: 1015 	Training Loss: 47.426921 	acc: 85.697888	Validation Loss: 265.898406, time5.5
Epoch: 1016 	Training Loss: 47.530227 	acc: 85.577365	Validation Loss: 265.071694, time5.5

Epoch: 1097 	Training Loss: 47.375268 	acc: 85.497016	Validation Loss: 266.144645, time5.5
Epoch: 1098 	Training Loss: 47.455617 	acc: 85.462580	Validation Loss: 264.827922, time5.4
Epoch: 1099 	Training Loss: 47.178222 	acc: 85.525712	Validation Loss: 265.685240, time5.5
Epoch: 1100 	Training Loss: 46.998393 	acc: 85.611800	Validation Loss: 267.194007, time5.4
Epoch: 1101 	Training Loss: 47.118916 	acc: 85.789715	Validation Loss: 268.112925, time5.5
Epoch: 1102 	Training Loss: 47.488139 	acc: 85.485537	Validation Loss: 267.032392, time5.4
Epoch: 1103 	Training Loss: 47.432660 	acc: 85.583104	Validation Loss: 265.522496, time5.5
Epoch: 1104 	Training Loss: 47.371442 	acc: 85.583104	Validation Loss: 267.958065, time5.4
Epoch: 1105 	Training Loss: 47.268136 	acc: 85.766758	Validation Loss: 266.243968, time5.5
Epoch: 1106 	Training Loss: 47.258571 	acc: 85.703627	Validation Loss: 265.434131, time5.4
Epoch: 1107 	Training Loss: 46.877870 	acc: 85.783976	Validation Loss: 266.682425, time5.5

Epoch: 1188 	Training Loss: 46.975436 	acc: 85.565886	Validation Loss: 269.010708, time5.4
Epoch: 1189 	Training Loss: 47.187787 	acc: 85.497016	Validation Loss: 269.592079, time5.4
Epoch: 1190 	Training Loss: 47.051959 	acc: 85.560147	Validation Loss: 269.275781, time5.5
Epoch: 1191 	Training Loss: 47.260484 	acc: 85.468320	Validation Loss: 268.735958, time5.4
Epoch: 1192 	Training Loss: 47.377181 	acc: 85.629017	Validation Loss: 270.183770, time5.5
Epoch: 1193 	Training Loss: 47.583792 	acc: 85.646235	Validation Loss: 269.524891, time5.4
Epoch: 1194 	Training Loss: 47.030915 	acc: 85.663453	Validation Loss: 268.312606, time5.5
Epoch: 1195 	Training Loss: 47.038567 	acc: 85.485537	Validation Loss: 270.475138, time5.4
Epoch: 1196 	Training Loss: 47.212657 	acc: 85.761019	Validation Loss: 268.719550, time5.5
Epoch: 1197 	Training Loss: 47.287267 	acc: 85.611800	Validation Loss: 267.770761, time5.4
Epoch: 1198 	Training Loss: 47.145699 	acc: 85.548669	Validation Loss: 267.277247, time5.5

KeyboardInterrupt: 

In [16]:
(output_cn==label_cn).sum()#/512

tensor(288, device='cuda:1')

In [14]:
len(output_cn)

512

In [5]:
output_cn

tensor([ 22,  65,  34, 104,  77,  10,  57,  36,  61,  26,  56, 119,  37,  51,
          6,   2, 120,  49,  12,  61, 116,  63,  83,  61,   9,   6,  18,  77,
         89,  60,  85,  22,  41,  18, 106,  47, 116,  81, 109,  65,   2,  21,
         56,  64,  91, 115,  28,   4,  70,  25,  28,  29,  71,  95,  72,  63,
         79, 115,  34,   7,  55,  15,  83,  76,  23,  93,  61,  39,  65,   3,
         22,  92, 113,  34,  68, 114,  78,  57,  42,   4,  29,  57,   9,  24,
         66, 117,  22,  17,  36,  76,  66,  10,  97,  74,  64,  86,  64,  49,
         54, 117,  77,  85,  93,  97,   3,  27,  68,  66,  20,  36,  66,  51,
         37,  78,  72,  73, 108,  73,  99,  72,   8,   8,  86,  66,  74,  61,
        116,   1,  48,  89,  56,  29,  27,  51,  20,  71,   8,  20,  11,  41,
        117,  61,  85,  85, 108,  80,  25,  83,  44, 102,  66,  38,  37, 115,
         36, 119,  72, 108,  37,  35,  89,  16,  48,  91, 118,  36, 114,  94,
        111,  83,  65,  39,  81,  94,  72, 110,  47,  10,  88, 1

In [6]:
label_cn

tensor([ 66,  65,  32, 104,  92, 112,  57,  88,  61,  26,  56,  51,  37,  96,
          6,   2, 120,  49,  12,  61, 116,  63,  89,  61,   9, 120,  18,  77,
         89,  60,  37,  56, 120,  18,  72,  56, 120,  81, 120,  65,   2,  21,
         56,  64,  66,  90,  28,   4,  70,  25,  28,  72,  50,  74,  72,  63,
         70,  17,  34,  85,  73,  86,  83,  76,  10,  93,  73,   5, 104,   3,
         22,  84,  98,   4,  68, 114,  66, 118,  16,   4,  29,  80, 106,  24,
         65, 117,  22,  98,  36,  90,  66,  10,  16,  24,  64,  86,  97,  92,
        112,  28, 101,  50, 114,  96,  58,  12,  68,  66,  20,  36,  66, 109,
         37,  78,  72,  68, 108,  73,  99,  72,   8,   8,  86,  66, 100,  92,
        116,   1,  48,  52,  56,   1, 102,  96,  12,  80,   8,  20,  60,  48,
         81, 108,  70,  82,  76,  80,  25,   9,  44,  96,  66,  38,   8, 115,
         36,  27,  72, 108,  37,  85,  89,  16,  48,  11, 100, 120, 114,  94,
         98,  10,  65,  39,  81,  42,  72, 110,  83,  98,  88, 1

In [3]:
torch.save({'model_state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()}, 'model_saved/mlp/multitask/last.pth')

In [3]:
loss_cn,loss_area1,loss_area2

(tensor(0.0336, device='cuda:1', grad_fn=<MeanBackward0>),
 tensor(0.0006, device='cuda:1', grad_fn=<MeanBackward0>),
 tensor(0.0080, device='cuda:1', grad_fn=<MeanBackward0>))

In [3]:
loss_cn,loss_area1,loss_area2

(tensor(0.6934, device='cuda:1', grad_fn=<MeanBackward0>),
 tensor(0.0074, device='cuda:1', grad_fn=<MeanBackward0>),
 tensor(0.4885, device='cuda:1', grad_fn=<MeanBackward0>))